In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [5]:
data = pd.read_csv("GOOG.csv", date_parser=True)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1321873955.py, line 1)

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
training_data = data[data['Date'] < '2022-10-01'].copy()
data_training = training_data.copy()

In [ ]:
training_data.tail()

In [ ]:
testing_data = data[data['Date'] >= '2022-10-01'].copy()

In [ ]:
testing_data

In [ ]:
training_data = training_data.drop(['Date', 'Adj Close'], axis = 1)

In [ ]:
training_data.head()

In [ ]:
sc = MinMaxScaler()
training_data= sc.fit_transform(training_data)
training_data

In [ ]:
X_train = []
y_train = []

In [ ]:
training_data.shape[0]

In [ ]:
for i in range(60, training_data.shape[0]):
    X_train.append(training_data[i - 60 : i])
    y_train.append(training_data[i, 0])
    
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

In [ ]:
model = Sequential()

In [ ]:
model.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 120, activation = 'relu'))
model.add(Dropout(0.2))


model.add(Dense(units = 1))


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
model.fit(X_train, y_train, epochs = 2, batch_size = 512)

In [ ]:
testing_data.head()

In [ ]:
past_60_days = data_training.tail(60)
testing_data = past_60_days.append(testing_data, ignore_index = True)
testing_data = testing_data.drop(['Date', 'Adj Close'], axis = 1)
testing_data.head()

In [ ]:
testing_data = sc.transform(testing_data)
testing_data

In [ ]:
X_test = []
y_test = []

for i in range(60, testing_data.shape[0]):
    X_test.append(testing_data[i - 60 : i])
    y_test.append(testing_data[i, 0])

X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
type(X_test)

In [ ]:
y_pred.shape

In [ ]:
sc.scale_

In [ ]:
scale = 1/6.69375383e-03
scale

In [ ]:
y_pred = y_pred*scale
y_test = y_test*scale

In [ ]:
# Visualising
plt.figure(figsize=(14,7))
plt.plot(y_test, color = 'black', label = 'Actual Price', linestyle='--')
plt.plot(y_pred, color = 'red', label = 'Predicted Price', linestyle=(5, (10, 2)))
plt.title('Google Stock Price Prediction using RNN')
plt.xlabel('Time')
plt.ylabel('Google\'s Stock Price')
plt.legend()
plt.show()